# Installing OpenCV

Install the openCV package and additional base libraries 

```
pip install opencv-contrib-python caer
```

# Reading Images and Videos

In [1]:
# importing openCV lib
import cv2 as cv

## reading image

In [6]:
def image_read(path):
    img = cv.imread(path) #reads image file as matrix
    try:
        cv.imshow(image_name, img) # shows image (args: window name, img_matrix)
        cv.waitKey(0)  # waits indefinitely 
    except:
        print("Read error")

In [8]:
image_name = input('Enter file name from "img" folder... ')
image_read(path=f'img/{image_name}')

Sometimes the images are too big that it can't fit the monitor window. OpenCV does not have a native way to deal with this problem. However, there are ways to resize an image to mitigate this issue. 

## Reading Videos
The `VideoCapture()` function takes two types of arguments as input. 
1. An __Int__ arg if you'd you use your camera as a source 
2. A __Str__ if the source is a file. 

In [9]:
def video_read(path):
    vdo = cv.VideoCapture(path)  

    while True:
        isTrue, frame = vdo.read()  # reads the file frame by frame. the read status is the isTrue flag
        try:
            cv.imshow(vdo_name, frame)

            #stopping from indefinite play
            if cv.waitKey(20) & 0xFF == ord('d'):
                break
        except KeyboardInterrupt:
            print("Ctrl + C detected...")
            break
        except:
            break

    vdo.release()          # releasing the capture pointer 
    cv.destroyAllWindows() # garbage collection 

In [10]:
vdo_name = input('Enter file name from "vdo" folder... ')
video_read(path = f'vdo\{vdo_name}')

# Rescaling images and videos.

The Resize and Rescale methods helps to reduce computational load while rendering an image or video. 

In [4]:
#define a rescale function

def rescaleFrame(frame, scale):
    height = int( frame.shape[0] * scale )   # scaling height and weight 
    width  = int( frame.shape[1] * scale )
    dim = (height, width)                    # wrapping into a tuple
    return cv.resize( frame, dim, interpolation=cv.INTER_AREA )  # resized frame 

Lets now modify the image and video read functions to allow resize. the arguments are as follows.
* __path__ : image or video path (STR)
* __scale__ : scalling factor (FLOAT)
* __side_by_side__ : if True both version will be shown, if False just the rescaled one. (BOOL)

## image read with resize 

In [5]:
def image_read_rescale(path, scale = 1, side_by_side = False):
    img = cv.imread(path) #reads image file as matrix
    img_scalled = rescaleFrame(img, scale=scale) #scaling
    
    try:
        if side_by_side:
            cv.imshow(image_name, img) # shows image (args: window name, img_matrix)
        cv.imshow(f'{image_name}(resized x{scale})', img_scalled)
        
        cv.waitKey(0)  # waits indefinitely 
    except:
        print("Read error")

In [6]:
image_name = input('Enter file name from "img" folder... ')
image_read_rescale(path=f'img/{image_name}', scale=0.5, side_by_side=True)

## Video read with resize

In [10]:
def video_read_rescale(path, scale, side_by_side):
    vdo = cv.VideoCapture(path)  

    while True: 
        isTrue, frame = vdo.read()  # reads the file frame by frame. the read status is the isTrue flag
        try:
            frame_scalled = rescaleFrame(frame, scale=scale)
            if side_by_side:
                cv.imshow(vdo_name, frame)
            cv.imshow(f'{vdo_name} (Rescaled x{scale})', frame_scalled)
            
            #stopping from indefinite play
            if cv.waitKey(20) & 0xFF == ord('d'):
                break
            if not isTrue:    # to prevent a read error 
                break
        except:
            break

    vdo.release()          # releasing the capture pointer 
    cv.destroyAllWindows() # garbage collection 

In [11]:
vdo_name = input('Enter file name from "vdo" folder... ')
video_read_rescale(path = f'vdo\{vdo_name}', scale=0.75, side_by_side=True)

## Reading Live feed 



In [1]:
def video_read_rescale_live(path, scale, side_by_side, live, res=None):
    '''
        path    : file path (str) for non-live and cam_id (int) for live feed
        scale   : scaler (float)
        side_by_side    : (bool) playing both original and scaled 
        live    : (bool) is live
        res     : (h,w) a tuple defining the live capture resolution (only for live)
    '''
    if live:
        print('Feed set to live')
        path = int(path)
    
    vdo = cv.VideoCapture(path)  

    if live and (res is not None):
        print(f'Resolution set to {res[0]}X{res[1]}')
        vdo.set(3,res[1])  # width
        vdo.set(4,res[0])  # height

    print('feeding...') 
    while True: 
        isTrue, frame = vdo.read()  # reads the file frame by frame. the read status is the isTrue flag
        try:
            frame_scalled = rescaleFrame(frame, scale=scale)
            if side_by_side:
                cv.imshow(vdo_name, frame)
            cv.imshow(f'{vdo_name} (Rescaled x{scale})', frame_scalled)
            
            #stopping from indefinite play
            if cv.waitKey(20) & 0xFF == ord('d'):
                break
            if not isTrue:    # to prevent a read error 
                break
        except:
            break

    vdo.release()          # releasing the capture pointer 
    cv.destroyAllWindows() # garbage collection 

In [6]:
vdo_name = input('Enter file name from "vdo" folder... ')
video_read_rescale_live( path = vdo_name, 
                    scale=0.75, 
                    side_by_side=True, 
                    live=True,
                    res = (800,600))

Feed set to live
Resolution set to 800X600
feeding...


# Draw Shape and write in an Image

## drawing on a blank image

In [3]:
import numpy as np

In [19]:
h,w = map(int, input('Enter res (w h): ').split())
blank_img = np.zeros(shape=(w,h, 3 ), dtype='uint8')    # create a blank image of size (w,h) with 3 col channels

print("Operations...")
print("\t [1] Fill color in (r,g,b) format")
print("\t [2] Fill a portion")
print("\t [3] draw rectangle")
print("\t [4] draw a circle")
choice = int(input("enter your choice... "))

# fill 
if choice == 1:
    r,g,b = map(int, input('Enter color code').split())
    blank_img[ : ] = r,g,b

# subfill
elif choice == 2:
    r,g,b = input('Enter color code').split()
    w1,w2,h1,h2 = map(int, input("Enter section (w1 w2 h1 h2)").split())
    blank_img[w1:w2 , h1:h2] = int(r),int(g),int(b)

# rectangle
elif choice == 3:
    r,g,b = map(int, input('Enter color code of the rectangle').split())
    w1,w2,h1,h2 = map(int, input("Enter section (w1 w2 h1 h2)").split())
    thk = int(input('Enter thickness... ')) # use thk = -1 to fill the reactangle 

    cv.rectangle(   img=blank_img, 
                pt1=(w1,h1), 
                    pt2=(w2,h2), 
                    color=(r,g,b), 
                    thickness=thk
                )

# Circle
elif choice == 4:
    r,g,b = map(int, input('Enter color code of the rectangle').split())
    cx,cy,r = map(int, input("Enter section (center_x, center_y, radius)").split())
    thk = int(input('Enter thickness... ')) # use thk = -1 to fill the reactangle 

    cv.circle(  img=blank_img,
                center=(cx,cy), 
                radius=r,
                color=(r,g,b), 
                thickness=thk
            )
else:
    print('wrong choice')


cv.imshow(f'Blank [{w}x{h}]', blank_img)                           # show the blank image 
cv.waitKey(0)

Operations...
	 [1] Fill color in (r,g,b) format
	 [2] Fill a portion
	 [3] draw rectangle
	 [4] draw a circle


-1

In [2]:
file_name  = input('Enter image name... ')
img = cv.imread(f'img/{file_name}')
cv.imshow(file_name, img)
cv.waitKey(0)

-1